<a href="https://colab.research.google.com/github/confusetrial/DeepLearningExercise/blob/main/%ED%95%9C%EA%B8%80_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%A0%95_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')
DATA_Path = '/content/drive/My Drive/Colab Notebooks/dataset/Korean movies/'

     |████████████████████████████████| 19.4MB 53.1MB/s 
     |████████████████████████████████| 92kB 5.5MB/s 
     |████████████████████████████████| 460kB 30.4MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Mounted at /content/drive


In [ ]:
#training data load
train_data = pd.read_csv(DATA_Path+'ratings_train.txt',header=0,delimiter='\t',quoting=3)
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
train_length = train_data['document'].astype(str).apply(len)
print('길이 중간값 :',np.median(train_length))

길이 중간값 : 27.0


In [ ]:
#단어 사용량 (단어 기준으로 불용어 제거해서 RNN 학습할 예정)
train_review = [review for review in train_data['document'] if type(review) is str]
train_word_count = train_data['document'].astype(str).apply(lambda x:len(x.split(' ')))
print('단어 사용량 중간값 :',np.median(train_word_count))

단어 사용량 중간값 : 6.0


In [ ]:
#텍스트에서 한글 성분만 가져오기
review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]",'',train_data['document'][0])
okt = Okt()
review_text=okt.morphs(review_text,stem=True)

In [ ]:
stopwords = set(['은','는','이','가','을','를','하','아','것','들','의','있','되','수','보','주','등','한'])
clean_review = [token for token in review_text if token not in stopwords]
print(clean_review)

['더빙', '진짜', '짜증나다', '목소리']


In [ ]:
#각각의 리뷰 전처리 하는 과정 : 한글 성분만 가져와서 문장으로 연결, 각 문장을 어절로 분해해서 불용어 제거 
def preprocessing(review,okt,using_stopwords=False,stop_words=[]):
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]",',',review)

    word_review = okt.morphs(review_text,stem = True)
    if using_stopwords:
        clean_review = [token for token in word_review if token not in stopwords]
    
    return clean_review

In [ ]:
#training data에 대한 리뷰 데이터 전처리
clean_review_data = []
for review in train_data['document']:
    if type(review) == str:
        clean_review_data.append(preprocessing(review,okt,True,stop_words=stopwords))
    else:
        clean_review_data.append([])

In [47]:
#test 데이터에 대한 리뷰 데이터 전처리
test_data = pd.read_csv(DATA_Path+'ratings_test.txt',header=0,delimiter='\t',quoting=3)

clean_test_review = []
for review in test_data['document']:
    if type(review) == str:
        clean_test_review.append(preprocessing(review,okt,True,stop_words=stopwords))
    else:
        clean_test_review.append([])
#데이터 tokenize 한 후 같은 길이로 맞춰줌, post-padding    
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_review_data)
train_seq = tokenizer.texts_to_sequences(clean_review_data)
test_seq = tokenizer.texts_to_sequences(clean_test_review)

word_vocab = tokenizer.word_index

MAX_LEN = 15

train_inputs = pad_sequences(train_seq,maxlen=MAX_LEN,padding='post')
train_labels = np.array(train_data['label'])
test_inputs = pad_sequences(test_seq,maxlen=MAX_LEN,padding='post')
test_label = np.array(test_data['label'])

In [49]:
#데이터 저장코드
np.save(open(DATA_Path+'train_input.npy','wb'),train_inputs)
np.save(open(DATA_Path+'train_label.npy','wb'),train_labels)
np.save(open(DATA_Path+'test_input.npy','wb'),test_inputs)
np.save(open(DATA_Path+'test_label.npy','wb'),test_label)

data_configs = {}
data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab)+1

json.dump(data_configs,open(DATA_Path+'data_configs.json','w'),ensure_ascii=False)

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout
from tensorflow.keras.optimizers import Adam

In [52]:
#RNN 모델 구성
model = Sequential()
model.add(Embedding(data_configs['vocab_size'],output_dim=128))
model.add(LSTM(256,return_sequences=True))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu',kernel_initializer='glorot_normal'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer=Adam(0.001),loss='binary_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 128)         5529728   
_________________________________________________________________
lstm_18 (LSTM)               (None, None, 256)         394240    
_________________________________________________________________
lstm_19 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_19 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                

In [53]:
history = model.fit(train_inputs,train_labels,batch_size=512,epochs=8,validation_split=0.3)

Epoch 1/8
206/206 [==============================] - 28s 116ms/step - loss: 0.5158 - acc: 0.7162 - val_loss: 0.3829 - val_acc: 0.8303
Epoch 2/8
206/206 [==============================] - 21s 100ms/step - loss: 0.3242 - acc: 0.8644 - val_loss: 0.3913 - val_acc: 0.8278
Epoch 3/8
206/206 [==============================] - 22s 108ms/step - loss: 0.2702 - acc: 0.8878 - val_loss: 0.4005 - val_acc: 0.8310
Epoch 4/8
206/206 [==============================] - 21s 101ms/step - loss: 0.2340 - acc: 0.9012 - val_loss: 0.4510 - val_acc: 0.8279
Epoch 5/8
206/206 [==============================] - 21s 104ms/step - loss: 0.1915 - acc: 0.9204 - val_loss: 0.5171 - val_acc: 0.8255
Epoch 6/8
206/206 [==============================] - 21s 101ms/step - loss: 0.1571 - acc: 0.9358 - val_loss: 0.5095 - val_acc: 0.8202
Epoch 7/8
206/206 [==============================] - 20s 99ms/step - loss: 0.1390 - acc: 0.9448 - val_loss: 0.5929 - val_acc: 0.8217
Epoch 8/8
206/206 [==============================] - 23s 112ms/

In [54]:
eval = model.evaluate(test_inputs,test_label)

1563/1563 [==============================] - 9s 6ms/step - loss: 0.6309 - acc: 0.8140
